In [1]:
from strategy import DualReallocationStrategy
import dffc
from dffc import HWOptimizer
from dffc import ReallocationBackTest as RBT
import json
import pandas as pd
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

## Download & Update Data

In [2]:
codes = ['007467', '004253']
names=['HL', 'GD']
start_date = '2019-07-01'
end_date = '2025-07-01'

try:
    fund_data = dffc.FundData.load('hlvsgd_all.pkl')
except FileNotFoundError:
    fund_data = dffc.FundData.download(
        codes,
        names=names,
        # start=start_date,
        end=end_date
    )

    fund_data.save('hlvsgd_all.pkl')

fund_data = fund_data.update()
price_data_org = fund_data.get('unit_value') # .dropna()

In [3]:
estimate_p = dffc.FundEstimateProvider()

price_data = dffc.append_estimates_to_prices(
    price_data_org,
    codes,
    estimate_p,
    target_timezone=None,
)

Estimate data fetched for 007467 on 2025-11-28 11:30:00+08:00
Estimate data fetched for 004253 on 2025-11-28 12:38:00+08:00
Added estimate rows for dates: 2025-11-28.


In [4]:
price_data

,007467,004253
date,,
2017-05-02 00:00:00+08:00,NaN,1.0725
2017-05-03 00:00:00+08:00,NaN,1.0705
2017-05-04 00:00:00+08:00,NaN,1.0555
2017-05-05 00:00:00+08:00,NaN,1.0543
2017-05-08 00:00:00+08:00,NaN,1.0529
...,...,...
2025-11-24 00:00:00+08:00,1.6641,3.2760
2025-11-25 00:00:00+08:00,1.6738,3.3305
2025-11-26 00:00:00+08:00,1.6647,3.3298


## Load HW Parameter

In [44]:
with open('../hw_opt_results/demo_hw.json', 'r') as f:
    opt_res = json.load(f)

## Backtest Strategy

In [45]:
single_strategy = DualReallocationStrategy(
    prices = price_data,
    threshold=0.6,
    adjust_factor=0.2,
    default_weights = 0.5,
    up_weights = 0.2,
    down_weights= 0.8,
    tolerance=0.01,
    hw_params_list = opt_res
)

In [46]:
single_bt = RBT(
    strategy=single_strategy,
    start_date="2022-07-01",
    end_date=None,
    initial_cash=100_000,
    fees=0.0,
    trade_delay=0
)

single_bt.run()

100%|██████████| 1/1 [00:00<00:00,  7.33it/s]


In [47]:
single_bt.stats()

Start                           2022-07-01 00:00:00+08:00
End                             2025-11-28 00:00:00+08:00
Period                                  832 days 00:00:00
Start Value                                      100000.0
End Value                                   192790.176477
Total Return [%]                                92.790176
Benchmark Return [%]                             74.52294
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                 7.351418
Max Drawdown Duration                    83 days 00:00:00
Total Trades                                          618
Total Closed Trades                                   616
Total Open Trades                                       2
Open Trade PnL                                9296.545733
Win Rate [%]                                    86.850649
Best Trade [%]                                  24.028208
Worst Trade [%

In [49]:
single_bt.plot()

In [13]:
single_strategy.hw

,007467,004253
date,,
2017-05-02 00:00:00+08:00,NaN,1.068871
2017-05-03 00:00:00+08:00,NaN,1.074937
2017-05-04 00:00:00+08:00,NaN,1.069308
2017-05-05 00:00:00+08:00,NaN,1.068094
2017-05-08 00:00:00+08:00,NaN,1.063885
...,...,...
2025-11-24 00:00:00+08:00,1.684591,3.306071
2025-11-25 00:00:00+08:00,1.682748,3.300348
2025-11-26 00:00:00+08:00,1.685003,3.293020


In [15]:
single_bt.plot(column = single_bt.pf.close.columns[1])

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b5a6f548-f80f-4f5f-ba2b-700665caace3',
              'x': array([Timestamp('2025-01-02 00:00:00+0800', tz='UTC+08:00'),
                          Timestamp('2025-01-03 00:00:00+0800', tz='UTC+08:00'),
                          Timestamp('2025-01-06 00:00:00+0800', tz='UTC+08:00'), ...,
                          Timestamp('2025-11-26 00:00:00+0800', tz='UTC+08:00'),
                          Timestamp('2025-11-27 00:00:00+0800', tz='UTC+08:00'),
                          Timestamp('2025-11-28 00:00:00+0800', tz='UTC+08:00')],
                         shape=(220,), dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('io7k8h/S+T8eFmpN8475P3UCmggbnv' ... 'p3nKL6P3L5D+m3r/o/f9k9eVio+j8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
       

## Hyperparameter Optimization

In [16]:
strategy = DualReallocationStrategy(
    prices = price_data,
    threshold= np.arange(0.3, 2.0, 0.01),
    adjust_factor= np.arange(0.1, 1.1, 0.1),
    default_weights = 0.5,
    up_weights = np.arange(0.1, 0.5, 0.1),
    down_weights= np.arange(0.6, 1.0, 0.1),
    tolerance=0.01,
    hw_params_list = opt_res
)

In [10]:
bt = RBT(
    strategy=strategy,
    start_date=val.index[0],
    end_date=val.index[-1],
    initial_cash=100_000,
    fees=0.00075,
    trade_delay=0
)

bt.run()

100%|██████████| 27200/27200 [05:27<00:00, 82.99it/s] 


In [13]:
bt.stats() # selected=1

,,,,,,,benchmark_return,total_return,sharpe_ratio,max_drawdown
param_group,threshold,adjust_factor,default_weights,up_weights,down_weights,tolerance,,,,
0,0.30,0.1,0.5,0.1,0.6,0.01,0.116117,0.121647,2.344893,-0.050749
1,0.30,0.1,0.5,0.1,0.7,0.01,0.116117,0.106384,2.122449,-0.043014
2,0.30,0.1,0.5,0.1,0.8,0.01,0.116117,0.091226,1.856769,-0.044434
3,0.30,0.1,0.5,0.1,0.9,0.01,0.116117,0.076176,1.559258,-0.045853
4,0.30,0.1,0.5,0.2,0.6,0.01,0.116117,0.115028,2.264215,-0.047347
...,...,...,...,...,...,...,...,...,...,...
27195,1.99,1.0,0.5,0.3,0.9,0.01,0.116117,0.115374,2.209050,-0.053519
27196,1.99,1.0,0.5,0.4,0.6,0.01,0.116117,0.115374,2.209050,-0.053519
27197,1.99,1.0,0.5,0.4,0.7,0.01,0.116117,0.115374,2.209050,-0.053519


In [7]:
bt.plot(index_levels=['threshold', 'adjust_factor'], column_levels=['up_weights', 'down_weights'])

In [12]:
# Map top indices to their parameter sets without using an unhashable Index as a dict key
best_params = bt.get_best_param(metric='total_return', top_n=5)
best_params

,param_group,threshold,adjust_factor,default_weights,up_weights,down_weights,tolerance,total_return
0,10065,0.92,1.0,0.5,0.1,0.7,0.01,0.247490
1,10064,0.92,1.0,0.5,0.1,0.6,0.01,0.247439
2,10066,0.92,1.0,0.5,0.1,0.8,0.01,0.247410
3,10067,0.92,1.0,0.5,0.1,0.9,0.01,0.247197
4,10048,0.92,0.9,0.5,0.1,0.6,0.01,0.246761


In [11]:
best_params= bt.get_weighted_best_params(
    metric_weight = {
        'total_return': 0.45,
        'sharpe_ratio': 0.45,
        'max_drawdown': 0.1
    },
    top_n=10
)

In [12]:
best_params

,param_group,threshold,adjust_factor,default_weights,up_weights,down_weights,tolerance,weighted_score,total_return,sharpe_ratio,max_drawdown
0,8496,0.83,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
1,9296,0.88,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
2,9616,0.90,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
3,7056,0.74,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
4,7856,0.79,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
5,7536,0.77,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
6,8336,0.82,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
7,9136,0.87,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
8,8016,0.80,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391
9,7216,0.75,0.2,0.5,0.1,0.6,0.01,0.950064,0.1652,2.986103,-0.053391


## Best Param Strategy

In [14]:
best_strategy = DualReallocationStrategy(
    prices = price_data,
    threshold = best_params['threshold'][0],
    adjust_factor = best_params['adjust_factor'][0],
    default_weights = 0.,
    up_weights = best_params['up_weights'][0],
    down_weights = best_params['down_weights'][0],
    tolerance=0.01,
    hw_params_list = opt_res
)

In [15]:
best_bt = RBT(
    strategy=best_strategy,
    start_date=test.index[0],
    # end_date="2025-07-02",
    initial_cash=20_000,
    fees=0.0,
    trade_delay=0
)

best_bt.run()

100%|██████████| 1/1 [00:00<00:00, 19.51it/s]


In [17]:
best_strategy.hw_signals

,007467,004253
date,,
2024-07-17 00:00:00+08:00,-0.556667,0.946667
2024-07-18 00:00:00+08:00,-0.384359,0.920133
2024-07-19 00:00:00+08:00,-0.607321,-0.670549
2024-07-22 00:00:00+08:00,-0.813953,-0.239203
2024-07-23 00:00:00+08:00,-0.850498,-0.677741
...,...,...
2025-10-23 00:00:00+08:00,0.816733,-0.399734
2025-10-24 00:00:00+08:00,0.493369,-0.864721
2025-10-27 00:00:00+08:00,0.506631,-0.949602


In [18]:
best_bt.pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-07-17 00:00:00+08:00,4811.16,2.0785,0.0,Buy
1,1,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-07-17 00:00:00+08:00,6259.78,1.5975,0.0,Buy
2,2,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-07-18 00:00:00+08:00,761.85,2.0788,0.0,Sell
3,3,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-07-18 00:00:00+08:00,987.36,1.6040,0.0,Buy
4,4,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-07-19 00:00:00+08:00,110.94,2.0243,0.0,Sell
5,5,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-07-19 00:00:00+08:00,141.08,1.5919,0.0,Buy
6,6,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-10-09 00:00:00+08:00,4871.34,1.5855,0.0,Sell
7,7,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-10-09 00:00:00+08:00,3656.62,2.1122,0.0,Buy
8,8,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 007467)",2024-10-10 00:00:00+08:00,1044.61,1.6480,0.0,Sell
9,9,"(0, 0.92, 0.8, 0.5, 0.1, 0.6, 0.01, 004253)",2024-10-10 00:00:00+08:00,812.19,2.1196,0.0,Buy


In [16]:
best_bt.stats()

Start                         2025-05-28 00:00:00+08:00
End                           2025-10-29 00:00:00+08:00
Period                                104 days 00:00:00
Start Value                                     20000.0
End Value                                  21840.455796
Total Return [%]                               9.202279
Benchmark Return [%]                           8.787323
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               4.747755
Max Drawdown Duration                  27 days 00:00:00
Total Trades                                         53
Total Closed Trades                                  51
Total Open Trades                                     2
Open Trade PnL                              1105.071704
Win Rate [%]                                  94.117647
Best Trade [%]                                10.753211
Worst Trade [%]                               -0

In [17]:
best_bt.plot()

In [18]:
best_bt.pf.orders.records_readable


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-05-28 00:00:00+08:00,7295.80,2.7413,0.0,Buy
1,1,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-05-29 00:00:00+08:00,145.90,2.7174,0.0,Sell
2,2,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 007467)",2025-05-29 00:00:00+08:00,242.51,1.6349,0.0,Buy
3,3,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-05-30 00:00:00+08:00,117.36,2.7379,0.0,Sell
4,4,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 007467)",2025-05-30 00:00:00+08:00,195.94,1.6399,0.0,Buy
...,...,...,...,...,...,...,...
98,98,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-10-27 00:00:00+08:00,435.49,3.2970,0.0,Buy
99,99,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 007467)",2025-10-28 00:00:00+08:00,766.79,1.6626,0.0,Sell
100,100,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 004253)",2025-10-28 00:00:00+08:00,401.36,3.1764,0.0,Buy
101,101,"(0, 0.83, 0.2, 0.0, 0.1, 0.6, 0.01, 007467)",2025-10-29 00:00:00+08:00,488.22,1.6492,0.0,Sell


In [19]:
delta_hdp = - best_strategy.hw_signals.iloc[:, 0] + best_strategy.hw_signals.iloc[:, 1]

In [20]:
best_strategy.hw_signals

,007467,004253
date,,
2025-05-28 00:00:00+08:00,0.335227,-0.446023
2025-05-29 00:00:00+08:00,0.232955,-0.877841
2025-05-30 00:00:00+08:00,0.293617,-0.273759
2025-06-03 00:00:00+08:00,0.690780,0.571631
2025-06-04 00:00:00+08:00,0.742210,0.399433
...,...,...
2025-10-23 00:00:00+08:00,0.819389,-0.776892
2025-10-24 00:00:00+08:00,0.498674,-0.928382
2025-10-27 00:00:00+08:00,0.509284,-0.954907


In [21]:
delta_hdp

date
2025-05-28 00:00:00+08:00   -0.781250
2025-05-29 00:00:00+08:00   -1.110795
2025-05-30 00:00:00+08:00   -0.567376
2025-06-03 00:00:00+08:00   -0.119149
2025-06-04 00:00:00+08:00   -0.342776
                               ...   
2025-10-23 00:00:00+08:00   -1.596282
2025-10-24 00:00:00+08:00   -1.427056
2025-10-27 00:00:00+08:00   -1.464191
2025-10-28 00:00:00+08:00   -1.480795
2025-10-29 00:00:00+08:00   -0.966887
Length: 104, dtype: float64